In [1]:
%load_ext memory_profiler
%load_ext line_profiler

In [2]:
import nimare as nim
from nimare import annotate

dset = nim.dataset.Dataset.load('/Users/tsalo/Desktop/ns-dataset/neurosynth_w_cogat.pkl.gz')
texts_df = dset.texts
texts_df = texts_df.reset_index(drop=False)

counts_df = annotate.text.generate_counts(
    texts_df, text_column='abstract', tfidf=False, max_df=0.1, min_df=0.05)
coordinates_df = dset.coordinates
print(counts_df.shape)
coordinates_df = coordinates_df.loc[:5000]

INFO:nimare.annotate.text:Retaining 14369/14371 studies


(14369, 207)


In [3]:
%mprun -T gclda_init -f annotate.gclda.GCLDAModel.__init__ model = annotate.gclda.GCLDAModel(counts_df, coordinates_df, mask=dset.masker.mask_img, n_topics=50, symmetric=True, n_regions=2, seed_init=1)

INFO:nimare.annotate.gclda:Constructing/Initializing GCLDA Model
INFO:nimare.annotate.gclda:IDs mismatch detected: retaining 140 of 14370 unique IDs




*** Profile printout saved to text file gclda_init. 


In [4]:
print(open('gclda_init', 'r').read())

Filename: /Users/tsalo/Documents/tsalo/NiMARE/nimare/annotate/gclda.py

Line #    Mem usage    Increment   Line Contents
    92   1246.2 MiB   1246.2 MiB       def __init__(
    93                                     self,
    94                                     count_df,
    95                                     coordinates_df,
    96                                     mask="mni152_2mm",
    97                                     n_topics=100,
    98                                     n_regions=2,
    99                                     symmetric=True,
   100                                     alpha=0.1,
   101                                     beta=0.01,
   102                                     gamma=0.01,
   103                                     delta=1.0,
   104                                     dobs=25,
   105                                     roi_size=50.0,
   106                                     seed_init=1,
   107                                 ):
   108

In [5]:
%mprun -T gclda -f annotate.gclda.GCLDAModel.fit model.fit(n_iters=1, loglikely_freq=1000, verbose=2)

INFO:nimare.annotate.gclda:Iter 0001: Sampling z
INFO:nimare.annotate.gclda:Iter 0001: Sampling y|r
INFO:nimare.annotate.gclda:Iter 0001: Updating spatial params




*** Profile printout saved to text file gclda. 


In [6]:
print(open('gclda', 'r').read())

Filename: /Users/tsalo/Documents/tsalo/NiMARE/nimare/annotate/gclda.py

Line #    Mem usage    Increment   Line Contents
   350   1168.7 MiB   1168.7 MiB       def fit(self, n_iters=10000, loglikely_freq=10, verbose=1):
   351                                     """
   352                                     Run multiple iterations.
   353                             
   354                                     Parameters
   355                                     ----------
   356                                     n_iters : :obj:`int`, optional
   357                                         Number of iterations to run. Default is 10000.
   358                                     loglikely_freq : :obj:`int`, optional
   359                                         The frequency with which log-likelihood is updated. Default value
   360                                         is 1 (log-likelihood is updated every iteration).
   361                                     verbose : {0, 1, 2}

In [7]:
%lprun -T gclda_time -f annotate.gclda.GCLDAModel._update_peak_assignments model.fit(n_iters=5, loglikely_freq=1000, verbose=2)

INFO:nimare.annotate.gclda:Iter 0002: Sampling z
INFO:nimare.annotate.gclda:Iter 0002: Sampling y|r
INFO:nimare.annotate.gclda:Iter 0002: Updating spatial params
INFO:nimare.annotate.gclda:Iter 0003: Sampling z
INFO:nimare.annotate.gclda:Iter 0003: Sampling y|r
INFO:nimare.annotate.gclda:Iter 0003: Updating spatial params
INFO:nimare.annotate.gclda:Iter 0004: Sampling z
INFO:nimare.annotate.gclda:Iter 0004: Sampling y|r
INFO:nimare.annotate.gclda:Iter 0004: Updating spatial params
INFO:nimare.annotate.gclda:Iter 0005: Sampling z
INFO:nimare.annotate.gclda:Iter 0005: Sampling y|r
INFO:nimare.annotate.gclda:Iter 0005: Updating spatial params



*** Profile printout saved to text file 'gclda_time'. 


In [8]:
print(open('gclda_time', 'r').read())

Timer unit: 1e-06 s

Total time: 3.54598 s
File: /Users/tsalo/Documents/tsalo/NiMARE/nimare/annotate/gclda.py
Function: _update_peak_assignments at line 467

Line #      Hits         Time  Per Hit   % Time  Line Contents
   467                                               def _update_peak_assignments(self, randseed):
   468                                                   """
   469                                                   Update y / r indicator variables assigning peaks->topics/subregions.
   470                                           
   471                                                   Parameters
   472                                                   ----------
   473                                                   randseed : :obj:`int`
   474                                                       Random seed for this iteration.
   475                                                   """
   476                                                   # Seed random num

In [9]:
counts_df

,10,12,20,ability,action,active,addition,additional,adults,affective,...,volume,volunteers,voxel,vs,word,words,working,working memory,years,young
id,,,,,,,,,,,,,,,,,,,,,
9065511-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9084599-1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
9114263-1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9185551-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9256495-1,0,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29779744-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29782510-1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29787789-1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
